# 0. Set VIdeo Name

In [47]:
def get_video_name(address): ### Pytube
    yt_address = YouTube(address)
    yt_filename = yt_address.filename
    yt_filename = yt_filename.replace("  ", " ")
    yt_filename = yt_filename.replace(" ", "_")
    yt_filename = yt_filename.replace("\xa0", "_")
    return yt_filename

# 1. Download Video

In [48]:
def get_video(address, filename): ### youtube_dl
    print(filename)
    ydl_opts = {'outtmpl':"./Videos/" + filename, 'format' : "bestvideo+bestaudio/best"}  
    with yt.YoutubeDL(ydl_opts) as ydl:
        ydl.download([address])

# 2. Check Video Fram per Second / Duration

In [3]:
import os
import subprocess

In [44]:
def get_fps_duration(filename):
    fps = 0
    video = cv2.VideoCapture("./Videos/" + filename + ".mkv");
    clip = VideoFileClip("./Videos/" + filename + ".mkv")

    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
    try:
        if int(major_ver)  < 3 :
            fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        else :
            fps = video.get(cv2.CAP_PROP_FPS)
        video.release();
    except OSError:
        process = subprocess.Popen(['/usr/bin/ffmpeg', '-i', "./Videos/"+filename+".webm", "-qscale", "0", "./Videos/"+filename+".mkv"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        clip = VideoFileClip("./Videos/" + filename + ".mkv")
    second = clip.duration
    global e_file
    e_file = int(second * fps)
    print("FPS : " + str(fps) + ", Second : " + str(second) + ", E_file : " + str(e_file))

# 3. Download All Video Frames

In [6]:
def video_to_frames(video, path_output_dir,filename):
    vidcap = cv2.VideoCapture("./Videos/" + video)
    print("🔥" + video)
    count = 0
    while vidcap.isOpened():
        if vidcap == False:
            print("vidcap is False")
        success, image = vidcap.read()
        if success:
            cv2.imwrite(os.path.join(path_output_dir, '%d.png') % count, image)
            count += 1
        else:
            break
    cv2.destroyAllWindows()
    vidcap.release()

# 4. Scene_Detect

In [60]:
class PySceneDetectArgs(object):
    def __init__(self, input, type='content'):
        self.input = input
        self.detection_method = type
        self.threshold = 5
        self.min_percent = 2000
        self.min_scene_len = 15
        self.block_size = 8
        self.fade_bias = 0
        self.downscale_factor = 1
        self.frame_skip = 0
        self.save_images = True
        self.start_time = None
        self.end_time = None
        self.duration = None
        self.quiet_mode = True
        self.stats_file = None

In [75]:
os.getcwd()

'/home/soma/SeulA/Content_Download'

In [76]:
def start_pyscenedetect(filename):
    print("💣 Start SceneDetect")
    video_path = "./Videos/" + filename + ".mkv"
    scene_detectors = scenedetect.detectors.get_available()
    smgr_content   = scenedetect.manager.SceneManager(PySceneDetectArgs(input=video_path, type='content'), scene_detectors)
    #smgr_threshold = scenedetect.manager.SceneManager(PySceneDetectArgs(input=video_path, type='threshold'), scene_detectors)
    scenedetect.detect_scenes_file(path=video_path, scene_manager=smgr_content)
    #scenedetect.detect_scenes_file(path=video_path, scene_manager=smgr_threshold)
    detected_scenes = smgr_content.scene_list

# 5. Main

In [9]:
from pytube import YouTube
import youtube_dl as yt
import cv2
from moviepy.editor import VideoFileClip
import os
from datetime import datetime
import scenedetect
import shutil

In [55]:
#video_list = ["https://www.youtube.com/watch?v=GeQSdm8XdHI"]

In [56]:
#video_list.append("https://www.youtube.com/watch?v=XSdCz8UcF4k")

In [68]:
#%store -r video_list

In [57]:
#print(video_list)

['https://www.youtube.com/watch?v=GeQSdm8XdHI']


In [77]:
for i in range(len(video_list)):
    filename = get_video_name(video_list[i])
    %store filename
    get_video(video_list[i], filename)
    get_fps_duration(filename)
    
    try:
        os.makedirs("./all_frames/"+filename)
    except:
        FileExistsError
    all_frames_dir = "./all_frames/" + filename + "/"
    all_frames = os.listdir(all_frames_dir)
    all_frames_cnt = len(all_frames)
    print(e_file)

    if all_frames_cnt == e_file:
        print("👍🏻 Already existing all frames.")
    else:
        video_to_frames(filename + ".mkv", all_frames_dir, filename)
        all_frames = os.listdir(all_frames_dir)
        all_frames_cnt = len(all_frames)
        if all_frames_cnt == e_file:
            print("🎉 Exactly downloaded video frame.")

    scene_list = []
    try:
        os.makedirs("./detect_frames/" + filename)
    except FileExistsError:
        pass
    start_pyscenedetect(filename)
    p_path = "."
    source = os.listdir(p_path)
    c_path = "./detect_frames/" + filename
    for files in source:
        if files.endswith('.jpg'):
            shutil.move(os.path.join(p_path,files), os.path.join(c_path,files))

Stored 'filename' (str)
Korean_One_Brand_tutorial_2_VDL_로드샵_원브랜드_메이크업_2_VDL_편_SSIN
[youtube] GeQSdm8XdHI: Downloading webpage
[youtube] GeQSdm8XdHI: Downloading video info webpage
[youtube] GeQSdm8XdHI: Extracting video information
[youtube] GeQSdm8XdHI: Downloading MPD manifest


[download] ./Videos/Korean_One_Brand_tutorial_2_VDL_로드샵_원브랜드_메이크업_2_VDL_편_SSIN.mkv has already been downloaded and merged
FPS : 29.97002997002997, Second : 596.66, E_file : 17881
17881
👍🏻 Already existing all frames.
💣 Start SceneDetect
